In [94]:
import os.path
from collections import defaultdict

from seleniumbase import Driver

driver = Driver()
driver.open("https://untappd.com/")

In [95]:
import dataclasses


@dataclasses.dataclass()
class BeerPriceSql:
    url: str
    name: str
    style: str
    abv: float
    ibu: int
    brewery: str
    rate: float
    volume: str
    price: float
    date_parsed: str
    venue: str

In [101]:
import re
from selenium.common import NoSuchElementException


def parse_beers(venue, venue_url, date_parsed):
    if driver.get_current_url() != venue_url:
        driver.open(venue_url)

    menu_items = driver.find_elements(".menu-item")

    beers = []
    for menu_item in menu_items:
        # 'https://untappd.com/b/augustine-avgustin-kin-dza-dza/6091216'
        href = menu_item.find_element("css selector", "a").get_attribute("href")
        # 'КИН-ДЗА-ДЗА'
        beer_name = menu_item.find_element("css selector", "a").text.strip()
        beer_name = re.sub("^\d+\.\s", "", beer_name)
        # 'Sour - Tomato / Vegetable Gose'
        emphasis_text = menu_item.find_element("css selector", "em").text.strip()
        # '4.9% ABV • 3 IBU • Augustine (Августин) •\n(3.92)'
        heading_text = menu_item.find_element("css selector", "h6").text.strip()
        # '0.5L Can\n268.00 RUB'

        volumes = []
        try:
            prices_text = menu_item.find_element("css selector", ".beer-prices").text.strip()
            volumes_and_prices_raw = list(filter(None, prices_text.splitlines()))
            for i in range(0, len(volumes_and_prices_raw), 2):
                try:
                    volume, price = volumes_and_prices_raw[i], volumes_and_prices_raw[i+1]
                except IndexError:
                    continue
                price = float(price.split()[0])
                volumes.append((volume, price))
        except NoSuchElementException:
            volumes = [("0", 0)]

        abv, ibu, brewery, rate = heading_text.split('•')
        abv = 0 if abv.startswith("N/A") else float(abv.split("%", 1)[0])

        parsed_ibu = ibu.strip().split()[0]
        ibu = 0 if parsed_ibu == 'N/A' else int(parsed_ibu)
        brewery = brewery.strip()
        parsed_rate = rate.strip().strip("()")
        rate = 0 if parsed_rate == 'N/A' else float(parsed_rate)

        for volume, price in volumes:
            beer = BeerPriceSql(
                url=href,
                name=beer_name,
                style=emphasis_text,
                abv=abv,
                ibu=ibu,
                brewery=brewery,
                rate=rate,
                volume=volume,
                price=price,
                date_parsed=date_parsed,
                venue=venue,
            )
            print(beer)
            beers.append(beer)

    print(f"Parsed {len(beers)}")
    return beers

In [97]:
from pathlib import Path
import os

db_path = Path(os.getcwd()).parent / "beer.db"

In [98]:
import sqlite3
from typing import List, NamedTuple


def insert_beers_to_db(beers: List[BeerPriceSql], db_path: str) -> None:
    # Подключаемся к базе данных
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Подготавливаем данные для вставки
    beer_data = [(
        beer.url,
        beer.name,
        beer.style,
        beer.abv,
        beer.ibu,
        beer.brewery,
        beer.rate,
        beer.volume,
        beer.price,
        beer.date_parsed,
        beer.venue,
    ) for beer in beers]

    # Вставляем данные
    cursor.executemany('''
        INSERT INTO untappd_beer_prices (url, name, style, abv, ibu, brewery, rate, volume, price, date_parsed, venue)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', beer_data)

    # Сохраняем изменения и закрываем соединение
    conn.commit()
    conn.close()

In [102]:
from typing import List, NamedTuple


class Venue(NamedTuple):
    name: str
    list_name: str
    list_url: str

venues = [
    # Venue("СуперХмель", "Bottle List", "https://untappd.com/v/superhmel/10587734"),
    # Venue('Eric the Red', 'Второй этаж', 'https://untappd.com/v/eric-the-red/3797427?menu_id=32466'),
    # Venue('Eric the Red', 'Первый этаж', 'https://untappd.com/v/eric-the-red/3797427?menu_id=32580'),
    # Venue('Eric the Red', '-1 этаж', 'https://untappd.com/v/eric-the-red/3797427?menu_id=32583'),
    # Venue('Eric the Red', 'Bottle List', 'https://untappd.com/v/eric-the-red/3797427?menu_id=4873'),
    # Venue('Share House', 'Bottles', 'https://untappd.com/v/share-house/7728362?menu_id=48054'),
    # Venue('Share House', 'Taps', 'https://untappd.com/v/share-house/7728362?menu_id=49451'),
    # Venue('Share House', 'Топовые забугорные', 'https://untappd.com/v/share-house/7728362?menu_id=199538'),
    # Venue('Craft Republic', 'Bottles', 'https://untappd.com/v/craft-republic/2180553?menu_id=6716'),
    # Venue('Craft Republic', 'Taps', 'https://untappd.com/v/craft-republic/2180553?menu_id=43397'),
    # Venue('Магазин - Бар "Имею Право!" Академическая', 'Taps', 'https://untappd.com/v/magazin-bar-imeyu-pravo-akademicheskaya/12588903'),
    # Venue('Магазин - Бар "Имею Право!" Академическая', 'Bottles', 'https://untappd.com/v/magazin-bar-imeyu-pravo-akademicheskaya/12588903?menu_id=215414'),
    # Venue('Магазин-Бар «Имею Право!»', 'Taps', 'https://untappd.com/v/magazinbar-imeyu-pravo/9395517'),
    # Venue('Магазин-Бар «Имею Право!»', 'Bottles', 'https://untappd.com/v/magazinbar-imeyu-pravo/9395517?menu_id=94686'),
    # Venue('Магазин-Бар «Имею Право!»', 'Sale', 'https://untappd.com/v/magazinbar-imeyu-pravo/9395517?menu_id=102981'),
    # Venue("RULE taproom", "TAP LIST", "https://untappd.com/v/rule-taproom/3363317"),
    Venue("RULE taproom", "BOTTLE/CAN LIST", "https://untappd.com/v/rule-taproom/3363317?menu_id=94708"),
]

In [103]:
import datetime
today = datetime.datetime.now().strftime("%Y-%m-%d")
print(today)

for venue in venues:
    beers = parse_beers(venue.name, venue.list_url, today)
    insert_beers_to_db(beers, db_path)

2025-02-19
BeerPriceSql(url='https://untappd.com/b/big-village-brewery-hypnic-jerk/6143216', name='HYPNIC JERK', style='Pale Ale - New England / Hazy', abv=6.0, ibu=35, brewery='Big Village Brewery', rate=4.1, volume='450ml Can', price=420.0, date_parsed='2025-02-19', venue='RULE taproom')
BeerPriceSql(url='https://untappd.com/b/brewdog-punk-ipa/5702', name='Punk IPA', style='IPA - American', abv=5.2, ibu=35, brewery='BrewDog', rate=3.71, volume='500ml Can', price=450.0, date_parsed='2025-02-19', venue='RULE taproom')
BeerPriceSql(url='https://untappd.com/b/hopfully-brewing-eyecatcher/5821059', name='Eyecatcher', style='Pale Ale - New England / Hazy', abv=4.8, ibu=0, brewery='Hopfully Brewing', rate=3.66, volume='440ml Can', price=580.0, date_parsed='2025-02-19', venue='RULE taproom')
BeerPriceSql(url='https://untappd.com/b/jaws-brewery-jaws-apa/480265', name='Jaws APA', style='Pale Ale - American', abv=5.2, ibu=43, brewery='Jaws Brewery', rate=3.87, volume='500ml Bottle', price=380.0,

In [79]:
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

beers_sql = cursor.execute("select * from untappd_beer_prices").fetchall()
beers = [BeerPriceSql(**sql) for sql in beers_sql]
beers[0]

BeerPriceSql(url='https://untappd.com/b/augustine-avgustin-kin-dza-dza/6091216', name='КИН-ДЗА-ДЗА', style='Sour - Tomato / Vegetable Gose', abv=4.9, ibu=3, brewery='Augustine (Августин)', rate=3.93, volume='0.5L Can', price=268.0, date_parsed='2025-02-18', venue='СуперХмель')

In [80]:
from collections import defaultdict
beer_map = defaultdict(lambda : defaultdict(list))
for beer in beers:
    beer_map[beer.name][beer.venue].append(f"{beer.volume} - {beer.price}")

In [85]:
venues = {beer.venue for beer in beers}
beer_table = [
    ("name", *(venue for venue in venues))
]
beer_table.extend(sorted([
    (beer_name, *('\n'.join(beer_venues.get(venue, [])) for venue in venues))
    for beer_name, beer_venues in beer_map.items()
]))
beer_table[:5]


[('name',
  'Магазин-Бар «Имею Право!»',
  'Craft Republic',
  'Магазин - Бар "Имею Право!" Академическая',
  'СуперХмель',
  'Share House',
  'Eric the Red'),
 ('#800 WBA', '', '330ml Bottle - 800.0', '', '', '', '330ml Bottle - 800.0'),
 ('1. Forty Five', '', '', '', '', '', '500ml Draft - 380.0'),
 ('1. Grape Mulled Mead', '', '', '1L - 760.0\n500ml - 380.0', '', '', ''),
 ('1. HYPNIC JERK', '', '400ml Draft - 390.0', '', '', '', '')]

In [88]:
from tabulate import tabulate

print(tabulate(beer_table, tablefmt="html"))

<table>
<tbody>
<tr><td>name                                                                             </td><td>Магазин-Бар «Имею Право!»</td><td>Craft Republic       </td><td>Магазин - Бар &quot;Имею Право!&quot; Академическая</td><td>СуперХмель           </td><td>Share House            </td><td>Eric the Red           </td></tr>
<tr><td>#800 WBA                                                                         </td><td>                         </td><td>330ml Bottle - 800.0 </td><td>                                         </td><td>                     </td><td>                       </td><td>330ml Bottle - 800.0   </td></tr>
<tr><td>1. Forty Five                                                                    </td><td>                         </td><td>                     </td><td>                                         </td><td>                     </td><td>                       </td><td>500ml Draft - 380.0    </td></tr>
<tr><td>1. Grape Mulled Mead                      